In [1]:
#Installing necessary dependences:
#!pip install -q keras-nlp

In [2]:
#!pip list


EnvironmentNameNotFound: Could not find conda environment: tf-gpu
You can list all discoverable environments with `conda info --envs`.




In [1]:
#imports:
import tensorflow as tf
import tensorflow_text as tf_text
import keras
import pandas as pd
import numpy as np
import os
import re
import string
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, MultiHeadAttention, Softmax, TextVectorization, LayerNormalization
from keras import Model, Input
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
import keras_nlp

Using TensorFlow backend


In [5]:
#tf.compat.v1.disable_eager_execution()

In [6]:
#Preprocessing Data
filepath = os.path.dirname(os.path.dirname(os.path.abspath("."))) + "/Datasets"
filepath = re.sub(r'\\', '/', filepath)
print(filepath)
X1 = pd.read_csv(filepath + "/Tatobea Project" + '/'+ "spa" +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
X1 = X1[X1.notnull().all(axis = 1)]

C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets


In [7]:
print(X1)

                                                  English  \
0                                                     Go.   
1                                                     Go.   
2                                                     Go.   
3                                                     Go.   
4                                                     Hi.   
...                                                   ...   
139008  A carbon footprint is the amount of carbon dio...   
139009  Since there are usually multiple websites on a...   
139010  If you want to sound like a native speaker, yo...   
139011  It may be impossible to get a completely error...   
139012  One day, I woke up to find that God had put ha...   

                                               Translated  
0                                                     Ve.  
1                                                   Vete.  
2                                                   Vaya.  
3                          

In [8]:
X1['Translated'] = "<ES> " + X1['Translated']

In [9]:
print(X1)

                                                  English  \
0                                                     Go.   
1                                                     Go.   
2                                                     Go.   
3                                                     Go.   
4                                                     Hi.   
...                                                   ...   
139008  A carbon footprint is the amount of carbon dio...   
139009  Since there are usually multiple websites on a...   
139010  If you want to sound like a native speaker, yo...   
139011  It may be impossible to get a completely error...   
139012  One day, I woke up to find that God had put ha...   

                                               Translated  
0                                                <ES> Ve.  
1                                              <ES> Vete.  
2                                              <ES> Vaya.  
3                          

In [10]:
train_x = pd.DataFrame()

In [11]:
for d in [x[0] for x in os.walk(filepath + "/DGT-TM")]:
    print(d)
    print(d[-6:])
    #Ignores TMX files
    if d[-1] == ")" or d[-6:] == "DGT-TM":
        continue
    #This code actually works, however due to the dataset it takes 1+ hour to run.
    #Because of this, at least for now I'll be restricting it to just the EN-ES folder.
    te = True
    if d[-5:] != "EN-ES":
        te = False
        continue
    
    fp = re.sub(r'\\', '/', d)
    E = fp.split("/")[-1].split("-")[0]
    print(E)
    T = fp.split("/")[-1].split("-")[1]
    print(T)
    print(d + '/'+ E + "-" + T +'.txt')
    
    temp = pd.read_csv(d + '/'+ E + "-" + T +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
    temp = temp[temp.notnull().all(axis = 1)]
    temp['Translated'] = "<" + T + "> " + temp['Translated']
    
    print(temp.head())
    if te:
        X1 = pd.concat([X1, temp])
    else:
        train_x = pd.concat([train_x, temp])


C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM
DGT-TM
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN -NL
EN -NL
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-BG
\EN-BG
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-CS
\EN-CS
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-DA
\EN-DA
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-DE
\EN-DE
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-EL
\EN-EL
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-ES
\EN-ES
EN
ES
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-ES/EN-ES.txt
                                             English  \
0            Commission Regulation (EC) No 1788/2004   
1                                 of 15 October 2004   
2  fixing the minimum selling prices for butter f...  

In [12]:
Y1 = pd.DataFrame()
Y1['Translated'] = X1.pop("Translated")
Y1['Translated'] = "[start] " + Y1['Translated'] + " [end]"

In [13]:
print(Y1)

                                                Translated
0                                   [start] <ES> Ve. [end]
1                                 [start] <ES> Vete. [end]
2                                 [start] <ES> Vaya. [end]
3                               [start] <ES> Váyase. [end]
4                                 [start] <ES> Hola. [end]
...                                                    ...
5696845         [start] <ES> Código país tercero [1] [end]
5696846     [start] <ES> Valor global de importación [end]
5696847  [start] <ES> Nomenclatura de países fijada por...
5696848  [start] <ES> El código «999» significa «otros ...
5696849  [start] <ES> DO L 337 de 24.12.1994, p. 66; Re...

[5835639 rows x 1 columns]


In [14]:
train_x1, test_x, train_y1, test_y = train_test_split(X1, Y1, test_size=0.5, random_state=42)

In [15]:
#val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=42)

In [16]:
try:
    train_y = train_x.pop("Translated")
except:
    #train_x doesn't have any portions, so we'll just describe it as a normal Dataframe
    train_y = pd.DataFrame()

In [17]:
train_x = pd.concat([train_x, train_x1])

In [18]:
train_y = pd.concat([train_y, train_y1])

In [19]:
del train_x1
del train_y1
del X1
del Y1
del temp

In [20]:
print(train_x)
print(train_y)
print(test_x)
print(test_y)

                                                   English
4569925  Repair and maintenance services of other profe...
5159865  If the policy-holder belongs to a group of und...
1200393                       Cash flow (% of total sales)
3355922                                 White mineral oils
3934638  Directive 2004/17/EC of the European Parliamen...
...                                                    ...
1431032                                         Division D
2095551                 Done at Brussels, 7 December 2012.
4787669  The project's strategy is grounded in SEESAC's...
4165720  In S.11.01 the 15th paragraph, second bullet o...
1553774                                           Whereas:

[2917819 rows x 1 columns]
                                                Translated
4569925  [start] <ES> Servicios de reparación y manteni...
5159865  [start] <ES> Si el tomador del seguro formara ...
1200393  [start] <ES> Flujo de caja (% de las ventas to...
3355922       [start] <ES> A

In [21]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2917819, 1)
(2917819, 1)
(2917820, 1)
(2917820, 1)


In [22]:
train_pairs = pd.concat([train_x['English'], train_y['Translated']], axis=1).values.tolist()
val_pairs = pd.concat([test_x['English'], test_y['Translated']], axis=1).values.tolist()
print(train_pairs[0])
print(train_pairs[1])

['Repair and maintenance services of other professional electrical equipment', '[start] <ES> Servicios de reparación y mantenimiento de otros equipos eléctricos profesionales [end]']
['If the policy-holder belongs to a group of undertakings for which consolidated accounts within the meaning of Directive 83/349/EEC (7) are drawn up, the criteria mentioned above shall be applied on the basis of the consolidated accounts.', '[start] <ES> Si el tomador del seguro formara parte de un conjunto de empresas cuyo balance consolidado se establece con arreglo a lo dispuesto en la Directiva 83/349/CEE (7), los criterios mencionados anteriormente se aplicarán sobre la base del balance consolidado. [end]']


In [23]:
#train_x = train_x.astype(str)
#test_x = test_x.astype(str)

In [24]:
#X = pd.concat([train_x, test_x])
#Y = pd.concat([train_y, test_y])
#Y_SP = Y[Y['Translated'].str.contains("<ES>")]

In [56]:
file = open('engvocab.txt', 'r', encoding="utf-8")
input_vocab = file.read().split("\n")
file.close()
input_vocab_size = len(input_vocab)

file = open('spavocab.txt', 'r', encoding="utf-8")
output_vocab = file.read().split("\n")
file.close()
output_vocab_size = len(output_vocab)

In [25]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
#eng_vectorization.adapt(train_eng_texts)
#spa_vectorization.adapt(train_spa_texts)

In [26]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [27]:
#STOP

In [28]:
#Actual Model:
#For now, let's start by creating an exact copy of the attention is all you need model.
#I tried for a long time to make a model using only the Keras Functional API,
#but it seems like I will need to create some custom layers in order to succeed.


In [29]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
    ])

In [30]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(EncoderLayer, self).__init__()


        self.embed_dim = d_model
        self.dense_dim = dff
        self.num_heads = num_attention_heads
        
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()


        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    
        self.supports_masking = True

    def call(self, inputs, mask=None):


        attention_output = self.attention(query=inputs, value=inputs, key=inputs)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [31]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
               *,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(DecoderLayer, self).__init__()

        # Masked multi-head self-attention.
        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )
        # Multi-head cross-attention.
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )

        # Point-wise feed-forward network.
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization.
        self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.add = tf.keras.layers.Add()  # instead of `+` to preserve mask

        # Dropout for the point-wise feed-forward network.
        #self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, encoder_outputs, mask=None):
        # The encoder output shape is `(batch_size, input_seq_len, d_model)`.

    
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, use_causal_mask=True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(self.add([out_2, proj_output]))


In [33]:
#Original hyperparameters
num_layers = 4
embed_dim = 256
latent_dim = 512
num_heads = 8
dropout_rate = 0.1
epochs=50
batch_size = 128
sequence_length = 256

In [34]:
#Dataset_train = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor)).batch(batch_size)
#Dataset_test = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor)).batch(batch_size)

In [35]:
#tf.compat.v1.disable_eager_execution()

In [36]:
#print(Dataset_train)
#print(Dataset_test)

NameError: name 'Dataset_train' is not defined

In [57]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, num_layers, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.enc_layers = [
        EncoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=dense_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.enc_layers[i](x, mask)
            x = self.enc_layers[i](x)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config






In [58]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


In [59]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, num_layers,  dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.dec_layers = [
        DecoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=latent_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.dec_layers[i](x, mask)
            x = self.dec_layers[i](x, encoder_outputs)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [61]:
print(input_vocab_size)

15001


In [63]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, input_vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, output_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x, encoded_seq_inputs)
x = tf.keras.layers.Dropout(0.5)(x)
decoder_outputs = tf.keras.layers.Dense(output_vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
#tf.compat.v1.enable_eager_execution()

In [64]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_4 (Positi  (None, None, 256)   3905792     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 256)   9469952     ['positional_embedding_

In [68]:
with tf.device('/GPU:1'):
    transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/50
    4/45591 [..............................] - ETA: 16:33:13 - loss: 4.3241 - accuracy: 0.0822

KeyboardInterrupt: 

In [70]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

[]


In [77]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [78]:
print(tf.__version__)

2.10.0


In [76]:
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

False
False


In [ ]:
#i = Input(shape=(max_length))

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#TLayer = TextVectorization(output_sequence_length = max_padding_size, vocabulary = m, output_mode = "int")
#inp = TextVectorization(output_sequence_length = max_padding_size)

In [ ]:
#inp = TLayer(i)

In [ ]:
#print(inp)

In [ ]:
#inp.adapt(train_x)

In [ ]:
#dir(inp)

In [ ]:
#print(inp.get_vocabulary())

In [ ]:
#f = open("textvectorvocap.txt", "w", encoding="utf-8")
#f.write("*".join(inp.get_vocabulary()))
#f.close()

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#print(m == inp.get_vocabulary())

In [ ]:
#print(inp.vocabulary_size())

In [ ]:
#train_x_vec = inp.predict(train_x)
#test_x_vec = inp.predict(test_x)

In [ ]:
#print(i.__call__())

In [ ]:
#em = Embedding(input_dim = max_length, output_dim = 512)(inp)

In [ ]:
#posen = keras_nlp.layers.SinePositionEncoding()(em)

In [ ]:
#mh1 = MultiHeadAttention(num_heads=3, key_dim=512, value_dim=512)

In [ ]:
#mho1 = mh1(posen, posen, posen)

In [ ]:
#no1 = LayerNormalization()
#ou1 = no1(mho1+posen)

In [ ]:
#fe1 = Dense(512)(ou1)
#ou2 = no1(ou1+fe1)

In [ ]:
#lin = Dense(tar_vocab_length)(ou2)

In [ ]:
#s = Softmax()(lin)

In [ ]:
#model = Model(inputs=i, outputs=s)
#print(model.summary())

In [ ]:
#Need to figure out metrics for language translation